## Image Segmentation アノテーション済みのpngファイル・colormapダウンロード

In [1]:
from abeja.datalake import Client as DatalakeClient
from abeja.datalake import APIClient

import urllib.request
import os
from tqdm import tqdm
import json
import re

### 設定
- OUTPUT_DATALAKE_CHANNEL_ID：annotation済みpngが格納されているDatalake Channel IDを指定してください
- ANNOTATION_JSON_NAME：Annotation ToolよりダウンロードしたJSONファイルをNotebookにアップロードしファイル名を指定してください
- download_folder_name：pngファイルおよびJSONファイルの格納先フォルダを作成するため、フォルダ名を入力してください

In [2]:
OUTPUT_DATALAKE_CHANNEL_ID = 'XXXXXXXXXX'
ANNOTATION_JSON_NAME = 'XXXXXXXX.json'
api_client = APIClient()
download_folder_name = 'XXXXXXXX'

In [3]:
datalake_client = DatalakeClient()
channel = datalake_client.get_channel(OUTPUT_DATALAKE_CHANNEL_ID)
os.mkdir(download_folder_name)

#def outputfile():

f = open(ANNOTATION_JSON_NAME,'r')
json_dict = json.load(f)

list_fileid = []
list_filename = []
list_ano_map = []

for x in tqdm(json_dict):
    anotation_data = x['information']
    origin_data = x['task']['metadata']
    
    origin_in = [d for d in origin_data if 'information' in d]
    origin_str = ','.join(map(str,origin_in))
    origin_name_pick = origin_str.split(':', 15)[10]
    origin_name = origin_name_pick.split(',', 2)[0]
    
    origin_id = re.sub(r"[ ,']", "", origin_name)
    
    ano_in = [s for s in anotation_data if 'is_combined' in s]
    list_str = ','.join(map(str,ano_in))
    file_id_pick = list_str.split(':', 2)[1]
    file_id_fix = file_id_pick.split(',', 2)[0]
    
    file_id = re.sub(r"[ ,']", "", file_id_fix)
    
    # Annotation済みpngファイルのDownload
    download_name = download_folder_name + '/' + origin_id
    
    file_download_url_map = api_client.get_channel_file_download(OUTPUT_DATALAKE_CHANNEL_ID, file_id)
    file_download_url = file_download_url_map['download_url']
        
    urllib.request.urlretrieve(file_download_url, download_name)
    
    # Colormap JSONファイルの作成
    ano_map = [m for m in anotation_data if 'color' in m]
    
    ano_map_list_str = ''.join(map(str,ano_map))
    
    ano_map_color = ano_map_list_str.split(',', 10)
    ano_map_label = ano_map_list_str.split(',', 4)[2]
    

    list_fileid.append(file_id)
    list_filename.append(origin_id)
    list_ano_map.append(ano_map)
    
   
 # Colormap JSONファイルのDownload
file_dict = dict(zip(list_filename,list_ano_map))
text = json.dumps(file_dict, indent=2)

with open(download_folder_name + '/colormap.json', 'w') as f:
    json.dump(file_dict, f)

    
print('Download OK')